# Shape-Removed

Here we remove the shape from the lesion images by scrambling image tiles, maintaining an equal balance between in-lesion and out-lesion tiles, and removing tiles containing both in-lesion and out-lesion area.

In [2]:
import cv2
import os
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

def crop_center(img,cropx):
    y,x,c = img.shape
    startx = x//2 - cropx//2
    starty = y//2 - cropx//2    
    return img[starty:starty+cropx, startx:startx+cropx, :]

def draw_grid(img, line_color=(0, 255, 0), thickness=1, type_=cv2.LINE_AA, pxstep=50):
    x = pxstep
    y = pxstep
    while x < img.shape[1]:
        cv2.line(img, (x, 0), (x, img.shape[0]), color=line_color, lineType=type_, thickness=thickness)
        x += pxstep

    while y < img.shape[0]:
        cv2.line(img, (0, y), (img.shape[1], y), color=line_color, lineType=type_, thickness=thickness)
        y += pxstep
    
    return img

def scramble_texture(img_name, IMG_SIZE, CROP_SIZE, RASTER_SIZE, basepath, seg_basepath):
    seg_path = os.path.join(seg_basepath, img_name)
    img_path = os.path.join(basepath, img_name)

    img = cv2.imread(img_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = crop_center(img, CROP_SIZE)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

    seg = cv2.imread(seg_path)
    seg = cv2.resize(seg, (IMG_SIZE, IMG_SIZE))
    seg = crop_center(seg, CROP_SIZE)
    seg = cv2.resize(seg, (IMG_SIZE, IMG_SIZE))

    # Rasterize image
    in_rasters = []
    out_rasters = []

    sqrt_num_rasters = IMG_SIZE//RASTER_SIZE
    num_rasters = sqrt_num_rasters*sqrt_num_rasters

    for idy in range(sqrt_num_rasters):
        for idx in range(sqrt_num_rasters):
            y_start = idy * RASTER_SIZE
            y_end = (idy + 1) * RASTER_SIZE
            x_start = idx * RASTER_SIZE
            x_end = (idx + 1) * RASTER_SIZE

            seg_raster = seg[y_start:y_end, x_start:x_end, :]

            u = np.unique(seg_raster)
            if len(u) != 1:
                pass
            else:
                raster = img[y_start:y_end, x_start:x_end, :]
                if u == 0:
                    out_rasters.append(raster)
                elif u == 255:
                    in_rasters.append(raster)

    new_image = np.zeros_like(img)

    if len(in_rasters) != 0 and len(out_rasters) != 0:
        # Scramble Textures
        for idy in range(sqrt_num_rasters):
            for idx in range(sqrt_num_rasters):
                id = idy + idx

                if id % 2 == 0:
                    # Inner
                    raster = random.choice(in_rasters)
                else:
                    # Outer
                    raster = random.choice(out_rasters)

                y_start = idy * RASTER_SIZE
                y_end = (idy + 1) * RASTER_SIZE
                x_start = idx * RASTER_SIZE
                x_end = (idx + 1) * RASTER_SIZE
                
                new_image[y_start:y_end, x_start:x_end, :] = raster
    else:
        # Scramble image
        for idy in range(sqrt_num_rasters):
            for idx in range(sqrt_num_rasters):
                raster = random.choice(in_rasters + out_rasters)

                y_start = idy * RASTER_SIZE
                y_end = (idy + 1) * RASTER_SIZE
                x_start = idx * RASTER_SIZE
                x_end = (idx + 1) * RASTER_SIZE
                
                new_image[y_start:y_end, x_start:x_end, :] = raster

    # Multiply with grid image to smoothen the hard edges
    if RASTER_SIZE == 8:
        line_thickness = 0
        kernel_size = 0
    elif RASTER_SIZE == 16:
        line_thickness = 2
        kernel_size = 4
    elif RASTER_SIZE == 32:
        line_thickness = 3
        kernel_size = 6

    if line_thickness != 0:
        img_grid = np.ones_like(new_image) * 255.0
        img_grid = draw_grid(img_grid, line_color=(0, 0, 0), thickness=line_thickness, type_=cv2.LINE_AA, pxstep=RASTER_SIZE)
        img_grid = cv2.blur(img_grid, (kernel_size, kernel_size))
        img_grid = img_grid / 255

        new_image = np.multiply(new_image, img_grid).astype(np.uint8)
    
    return new_image

In [3]:
lst_settings = [
    {
        'IMG_SIZE': 256,
        'CROP_SIZE': 200,
        'RASTER_SIZE': 16,
    }
]

for setting in lst_settings:
    IMG_SIZE = setting['IMG_SIZE']
    CROP_SIZE = setting['CROP_SIZE']
    RASTER_SIZE = setting['RASTER_SIZE']
    DEST_FOLDER = f'Derm7pt/augmentations/shape-removed_IMGSize-{IMG_SIZE}_RasterSize-{RASTER_SIZE}_CropSize-{CROP_SIZE}'

    data = pd.read_csv('Derm7pt/original/meta/meta.csv')
    data = data['derm']

    for img_name in data:

        img = scramble_texture(
            img_name, 
            IMG_SIZE=IMG_SIZE, 
            CROP_SIZE=CROP_SIZE, 
            RASTER_SIZE=RASTER_SIZE,
            basepath='Derm7pt/original/images',
            seg_basepath='Derm7pt/segmentations/images'
            )

        # Save processed image
        basedir = img_name.split('/')[0]
        basedir = os.path.join(DEST_FOLDER, basedir)
        current_filepath = os.path.join(DEST_FOLDER, img_name)
                
        if not os.path.exists(basedir):
            os.makedirs(basedir)
        
        cv2.imwrite(current_filepath, img)

In [3]:
lst_settings = [
    {
        'IMG_SIZE': 256,
        'CROP_SIZE': 200,
        'RASTER_SIZE': 16,
    }
]

for setting in lst_settings:
    IMG_SIZE = setting['IMG_SIZE']
    CROP_SIZE = setting['CROP_SIZE']
    RASTER_SIZE = setting['RASTER_SIZE']
    DEST_FOLDER = f'ISIC/augmentations/shape-removed_IMGSize-{IMG_SIZE}_RasterSize-{RASTER_SIZE}_CropSize-{CROP_SIZE}'

    data = pd.read_csv('ISIC/csvs/multi-class_filtered.csv')
    data = data['image']

    for img_name in data:

        img_name = '/'.join(img_name.split('/')[6:])

        img = scramble_texture(
            img_name, 
            IMG_SIZE=IMG_SIZE, 
            CROP_SIZE=CROP_SIZE, 
            RASTER_SIZE=RASTER_SIZE,
            basepath='ISIC/original',
            seg_basepath='ISIC/segmentations'
            )

        # Save processed image
        basedir = '/'.join(img_name.split('/')[:-1])
        basedir = os.path.join(DEST_FOLDER, basedir)
        current_filepath = os.path.join(DEST_FOLDER, img_name)       
                
        if not os.path.exists(basedir):
            os.makedirs(basedir)
        
        cv2.imwrite(current_filepath, img)

